<div class="alert alert-block alert-info" >
    <h1>Robot Perception</h1>
    <h3>General Information:</h3>
    <p>Please do not add or delete any cells. Answers belong into the corresponding cells (below the question). If a function is given (either as a signature or a full function), you should not change the name, arguments or return value of the function.<br><br> If you encounter empty cells underneath the answer that can not be edited, please ignore them, they are for testing purposes.<br><br>When editing an assignment there can be the case that there are variables in the kernel. To make sure your assignment works, please restart the kernel and run all cells before submitting (e.g. via <i>Kernel -> Restart & Run All</i>).</p>
    <p>Code cells where you are supposed to give your answer often include the line  ```raise NotImplementedError```. This makes it easier to automatically grade answers. If you edit the cell please outcomment or delete this line.</p>
    <br>The server resource is limited to 2 core cpu and 1GB RAM at max per user. If you use more than that, the kernel may die. Nevertheless, you can bring it up again by restaring the kernel (Kernel -> Restart and clear output).<br>
    <h3>Submission:</h3>
    <p>Please submit your notebook via the web interface (in the main view -> Assignments -> Submit). The assignments are due on <b>Monday at 0:00.</b> (i.e. Sunday 23:59 + 1 min)</p>
    <h3>Group Work:</h3>
    <p>You are allowed to work in groups of up to two people. Please enter the UID (your username here) of each member of the group into the next cell. We apply plagiarism checking, so do not submit solutions from other people except your team members. If an assignment has a copied solution, the task will be graded with 0 points for all people with the same solution.</p>
    <p><b>YOU SHOULD ONLY SUBMIT EXACTLY ONE PER GROUP</b></p>
    <h3>Questions about the Assignment:</h3>
    <p>If you have questions about the assignment please post them in the LEA forum before the deadline. Don't wait until the last day to post questions.</p></div>

<div class="alert alert-block alert-danger">
<p><b>Put your answer in the PROVIDED CELLS only!</b></p>
<b>Any new cell is not visible during the grading.</b>
<p>There is one code and markdown cell available for each question, so that you can use one or both of them according to the type of the question.</p>
<p>Do not copy the metadata from one cell to another, it is unique to that cell only.</p></div>

In [ ]:
'''
Group Work:
Enter the UID (i.e. student2s) of each team member into the variables. 
If you work alone please leave the second variable empty.
'''
member1 = ''
member2 = ''

In [ ]:
'''
Put here the average solution time for the full exercise set in estimated MINUTES".
AverageSolutionTimeinMinutes should be > or < 4711
i.e.
AverageSolutionTimeinMinutes = 180
'''

AverageSolutionTimeinMinutes = 4711

In [ ]:
# This is a test cell, please ignore it!

 # Task 1 : Pose three questions (and ans.) to last lecture [1 point]
 The format of the question and answer should be [Q1,A1,Q2,A2,Q3,A3], where Q1 is the question and A1 is the answer.
Both Q and A should contain equal to or more than three words. Do not remove any markdown tag like $<$br$>$ in the answer cell.

Q1 = ...
<br>
A1 = ...
<br>
<br>
Q2 = ...
<br>
A2 = ...
<br>
<br>
Q3 = ...
<br>
A3 = ...

In [ ]:
# This is a test cell, please ignore it!

# Task 2: Read parts of chapter 4 (Estimation – 2D Projective Transformations) of MVG namely 4.1, 4.4., 4.6., 4.7., 4.8. [1 point]<br>Produce some sort of a summary

YOUR ANSWER HERE

# Task 3: Implement Algor 4.2 (normalized DLT) in python by HAND, don't use the in-build image tool boxes. [3 points]

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

# Task 4: Validate your results using the test images and by comparison TO the in build image library methods! Use the images found in the "./boat/" sub-dir (or any other you pick: needed: 2 images and two lists of corresponding point pair coordinates,pls provide MORE test cases)[1 point]<br>NB I am not 100% sure that the "boat" list is matching..)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE